In [54]:
import re
import csv
import json
import operator
import numpy as np
from watson_developer_cloud import NaturalLanguageClassifierV1

In [12]:
data = np.load("fullsetcompressed.npz",encoding='latin1')
data = data.f.arr_0

In [14]:
trainingdata = data[0:100]
testdata = data[100:200]

In [48]:
def buildstoplist():
    index = {}
    wordcount = 0
    wordprob = {}
    stopwords = []
    for i in trainingdata:
        #for char in i[0]:
        #    char = re.sub('[^a-zA-Z0-9 \n\.]', '', char)
        for word in i[0].split():
            word = re.sub('[^a-zA-Z0-9 \n\.]', '', word)
            if word != '':
                wordcount += 1
                if word not in index:
                    index[word] = 1
                else:
                    index[word] += 1
    
    for word in index.iterkeys():
        wordprob[word] = round(float(index[word])/wordcount*100,4)
        
    stoplist = sorted(wordprob.iteritems(), key=operator.itemgetter(1) ,reverse=True)[:20]
    
    for i in stoplist:
        stopwords.append(i[0])
    
    return stoplist, stopwords

In [49]:
stoplist,stopwords = buildstoplist()

In [50]:
stopwords

['the',
 'return',
 'if',
 'to',
 'import',
 'for',
 'public',
 'a',
 'is',
 'in',
 'of',
 'this',
 'new',
 'var',
 'and',
 'or',
 'file',
 'License',
 'private',
 'final']

In [16]:
def removestopwords(line, stoplist):
    pattern = re.compile(r'\b(' + r'|'.join(stopwords) + r')\b\s*')
    return pattern.sub('', line)

In [51]:
chunk = 1024

with open('stopwords.csv', 'w') as csvfile:
    fieldnames = ['text', 'pl']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    for i in trainingdata:
        for j in range(0,len(i[0]),chunk):
            text = re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i[0][j:j+chunk]))))   
            if text == '':
                continue
            else:
                writer.writerow({'text': text, 'pl': i[1]})
                
with open('nostopwords.csv', 'w') as csvfile:
    fieldnames = ['text', 'pl']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    for i in trainingdata:
        processedtext = removestopwords(i[0],stopwords)
        for j in range(0,len(processedtext),chunk):
            text = re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,processedtext[j:j+chunk]))))   
            if text == '':
                continue
            else:
                writer.writerow({'text': text, 'pl': i[1]})

In [52]:
username="c01269a1-fe44-49be-94ba-6f257c263911"
password="Xe2LNum2uHwM"

classifier_id = "pynotpy-natural-la-1510788551714"
natural_language_classifier = NaturalLanguageClassifierV1(
    username=username,
    password=password)

In [63]:
with open('stopwords.csv', 'rb') as stopwords:
    print(json.dumps(natural_language_classifier.create(training_data=stopwords, name='stopwords'), indent=2))
    
with open('nostopwords.csv', 'rb') as nostopwords:
    print(json.dumps(natural_language_classifier.create(training_data=nostopwords, name='nostopwords'), indent=2))

{
  "status": "Training", 
  "name": "stopwords", 
  "language": "en", 
  "created": "2018-03-02T21:08:40.480Z", 
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/8fbfc9x295-nlc-3659", 
  "status_description": "The classifier instance is in its training phase, not yet ready to accept classify requests", 
  "classifier_id": "8fbfc9x295-nlc-3659"
}
{
  "status": "Training", 
  "name": "nostopwords", 
  "language": "en", 
  "created": "2018-03-02T21:08:41.733Z", 
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/8fc193x296-nlc-3698", 
  "status_description": "The classifier instance is in its training phase, not yet ready to accept classify requests", 
  "classifier_id": "8fc193x296-nlc-3698"
}


In [68]:
natural_language_classifier.status("8fc193x296-nlc-3698")

{u'classifier_id': u'8fc193x296-nlc-3698',
 u'created': u'2018-03-02T21:08:41.733Z',
 u'language': u'en',
 u'name': u'nostopwords',
 u'status': u'Available',
 u'status_description': u'The classifier instance is now available and is ready to take classifier requests.',
 u'url': u'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/8fc193x296-nlc-3698'}

In [65]:
def compute_watson_accuracy(testdata, classifier_id):
    count = 0
    for i in testdata:
        x = natural_language_classifier.classify(classifier_id,re.sub(' +',' '," ".join(re.split(r'[^\w]', re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,i[0]))))[0:1024])
        if x['top_class'] == i[1]:
            count += 1
    return float(count)/len(testdata)

In [69]:
acc1 = compute_watson_accuracy(testdata,"8fbfc9x295-nlc-3659")
acc2 = compute_watson_accuracy(testdata,"8fc193x296-nlc-3698")

In [71]:
print acc1, acc2

0.85 0.86
